# RAG Chatbot for Primary School Students
## Knowledge Base: Cells and Chemistry of Life

This notebook demonstrates a complete RAG (Retrieval-Augmented Generation) chatbot system using:
- **Local LLM**: Ollama Gemma3:4b
- **Knowledge Base**: Cells and Chemistry of Life.pdf  
- **Vector Store**: FAISS with sentence transformers
- **Evaluation**: RAGAS framework with MLflow tracking
- **Target Audience**: Primary school students (ages 6-12)

In [1]:
import sys
import os
sys.path.append('src')

from dotenv import load_dotenv
load_dotenv()

from src import Config, DocumentProcessor, VectorStore, RAGChatbot, RAGASEvaluator
import mlflow
import mlflow.sklearn
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("All modules imported successfully!")

# Validate API key availability
config = Config()
api_key_available = config.validate_api_key()

if api_key_available:
    print("OpenAI API key loaded successfully!")
else:
    print("WARNING: OpenAI API key not found - RAGAS evaluation will fail")
    print("Please check your .env file")

# Initialize MLflow
mlflow.set_experiment("RAG_Chatbot_RAGAS_Evaluation")
print("MLflow experiment initialized!")

print(f"Looking for PDF: {config.DATA_DIR}/{config.PDF_FILE}")

All modules imported successfully!
OpenAI API key loaded successfully!
MLflow experiment initialized!
Looking for PDF: data/Cells and Chemistry of Life.pdf


## 2. Document Processing
Load and process the "Cells and Chemistry of Life.pdf" document

In [2]:
# Initialize document processor and load PDF
doc_processor = DocumentProcessor(config)

chunks = doc_processor.process_document()

if chunks:
    print(f"\n Document Processing Results:")
    print(f"   Total chunks: {len(chunks)}")
    print(f"   Average chunk size: {sum(len(c['content']) for c in chunks) // len(chunks)} characters")
    print(f"\n Sample chunk:")
    print(f"   {chunks[0]['content'][:200]}...")
else:
    print("No chunks created. Please check if the PDF file exists in the data/ directory.")

Successfully loaded PDF: 109283 characters
Created 289 chunks

 Document Processing Results:
   Total chunks: 289
   Average chunk size: 384 characters

 Sample chunk:
   2 Chapter 1 Cells and the Chemistry of LifeCell Structure and OrganisationChapter 1 Imagining Cells as Chemical Factories All living things are made up of billions of tiny cells, just as a building is...


## 3. Vector Store Setup
Create embeddings and build FAISS index for similarity search

In [3]:
# Initialize vector store
vector_store = VectorStore(config)

# Try to load existing index, otherwise create new one
if not vector_store.load_index():
    print("Creating new vector index...")
    vector_store.setup_from_chunks(chunks)
    vector_store.save_index()

print(f"\nVector Store Ready!")
print(f" Index size: {vector_store.index.ntotal} vectors")
print(f" Embedding dimension: {vector_store.dimension}")

Loading embedding model...
Embedding model loaded. Dimension: 384
Index loaded from faiss_index.bin

Vector Store Ready!
 Index size: 289 vectors
 Embedding dimension: 384


## 4. RAG Chatbot Initialization
Initialize the chatbot with Ollama integration

In [4]:
try:
    chatbot = RAGChatbot(vector_store, config)
    print("RAG Chatbot initialized successfully!")
    print(f"Model: {config.OLLAMA_MODEL}")
    print(f"Similarity threshold: {config.SIMILARITY_THRESHOLD}")
    print(f"Top-K retrieval: {config.TOP_K_DOCS}")
except Exception as e:
    print(f"Failed to initialize chatbot: {str(e)}")

Ollama connected - gemma3:4b available
RAG Chatbot initialized successfully!
Model: gemma3:4b
Similarity threshold: 0.5
Top-K retrieval: 5


## 6. Custom Question Testing
Ask your own questions to the chatbot

In [ ]:
# Interactive question function
def ask_question(question: str):
    """Ask a question to the chatbot and display results"""
    result = chatbot.answer(question)
    
    print(f"Question: {question}")
    print(f"Answer: {result['answer']}")
    print(f"Details: In scope: {result['in_scope']} | Confidence: {result['confidence']:.3f}")
    
    # return result

ask_question("What is the main function of mitochondria in a cell?")

ask_question("How does the structure of a red blood cell help it transport oxygen?")

ask_question("What is diffusion?")

ask_question("How is the rough endoplasmic reticulum (RER) involved in protein transport?")

ask_question("What structures are found only in plant cells and not in animal cells?")

Question: What is the main function of mitochondria in a cell?
Answer: That’s a great question! Mitochondria are like the cell’s tiny powerhouses. They help the cell make energy, kind of like how you need energy to run and play!
Details: In scope: True | Confidence: 0.567
Question: How does the structure of a red blood cell help it transport oxygen?
Answer: That’s a great question! Red blood cells are really clever! They have a special shape – like a little donut – which makes them super good at carrying oxygen around your body. This shape gives them lots of space to hold onto oxygen, like a little backpack!
Details: In scope: True | Confidence: 0.553
Question: What is diffusion?
Answer: Diffusion is like when you smell yummy cookies baking! It’s when tiny bits of something, like the perfume, spread out from where there’s lots of them to where there are fewer. The smell travels until it’s spread out evenly – that’s diffusion!
Details: In scope: True | Confidence: 0.666
Question: How is

{'query': 'What structures are found only in plant cells and not in animal cells?',
 'answer': 'Okay, great question! \n\nOnly plant cells have a cell wall, which is like a strong fence around the cell to keep it in shape and protect it. They also have chloroplasts, which are like tiny kitchens that make food for the plant using sunlight!',
 'in_scope': True,
 'confidence': 0.6687755465507508,
 'chunks_used': 5}

## 7. RAGAS Evaluation with MLflow Tracking
Evaluate the chatbot performance using RAGAS metrics and track with MLflow

In [6]:
# Initialize RAGAS evaluator
evaluator = RAGASEvaluator(chatbot)

# Use sample test cases
test_cases = evaluator.create_sample_test_cases()

print("Test cases for evaluation:")
for i, case in enumerate(test_cases, 1):
    print(f"   {i}. {case['question']}")

print(f"\nRunning RAGAS evaluation with {len(test_cases)} test cases...")
scores = evaluator.evaluate_with_mlflow(test_cases, len(chunks))
evaluator.print_evaluation_report(scores, config.TARGET_RAGAS_SCORE)

print("Evaluation complete! Run 'mlflow ui' to view results.")

Test cases for evaluation:
   1. What is the main function of mitochondria in a cell?
   2. How does the structure of a red blood cell help it transport oxygen?
   3. What is diffusion?
   4. How is the rough endoplasmic reticulum (RER) involved in protein transport?
   5. What structures are found only in plant cells and not in animal cells?

Running RAGAS evaluation with 5 test cases...


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

MLflow tracking successful!

RAGAS EVALUATION REPORT
FAIL faithfulness: 0.520
PASS answer_relevancy: 0.935
FAIL context_precision: 0.684
PASS context_recall: 1.000

FAIL Overall Score: 0.785
Target Score: 0.8
Target not reached. Consider:
   - Better quality prompts
   - Improved chunking strategy
   - Different retrieval parameters
Evaluation complete! Run 'mlflow ui' to view results.
